In [5]:
import random
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from deep_translator import GoogleTranslator

#проверяем доступность сайта
req = Request(
    url='https://odds.ru/football/forecasts/',
    headers={'User-Agent': 'Mozilla/5.0'}
)
website_url = urlopen(req).read()
soup = BeautifulSoup(website_url, 'lxml')

links = []
news_link = soup.find_all('div', class_='forecasts-post__title')
for news in news_link:
    links.append(
        {
            'link': 'https://odds.ru' + news.find('a').get('href'),
            'text': news.find('a').get_text()#.lower().replace('«', '').replace('»', '').replace('— ', '')
        })
df1 = pd.DataFrame(links)
df1

,link,text
0,https://odds.ru/football/forecasts/elche-espan...,«Эльче» - «Эспаньол» - прогноз на матч 22-го т...
1,https://odds.ru/football/forecasts/prognoz-bay...,Прогноз «Байер» - «Майнц» - матч Бундеслиги
2,https://odds.ru/football/forecasts/borussiya-d...,«Боруссия» Дортмунд - «Герта»: прогноз и превь...
3,https://odds.ru/football/forecasts/prevyu-unio...,Превью «Унион Берлин» - «Шальке» - матч Бундес...
4,https://odds.ru/football/forecasts/osasuna-rea...,«Осасуна» - «Реал Мадрид» - прогноз на матч 22...
5,https://odds.ru/football/forecasts/manchester-...,«Манчестер Юнайтед» - «Лестер Сити»: прогноз н...
6,https://odds.ru/football/forecasts/adelaida-yu...,«Аделаида Юнайтед» - «Вестерн Сидней Уондерерс...
7,https://odds.ru/football/forecasts/malorka-vil...,«Мальорка» - «Вильярреал» - прогноз на матч 22...
8,https://odds.ru/football/forecasts/prognoz-ben...,Прогноз «Бенфика» - «Боавишта»: матч 21-го тур...
9,https://odds.ru/football/forecasts/betis-valya...,«Бетис» - «Вальядолид» - прогноз на матч 22-го...


In [6]:
df2 = pd.DataFrame()
key_words = ['Специя', 'Ювентус']
key_words = [item.lower() for item in key_words]
df1.text = [item.lower() for item in df1.text]
for line in df1.text:
    if all(word in line for word in key_words):
        df0 = df1[df1.text.str.contains(line)]
        df2 = pd.concat([df2, df0], axis= 0 )
df2

,link,text
15,https://odds.ru/football/forecasts/prevyu-spec...,превью «специя» - «ювентус»: матч 23-го тура с...


In [3]:
#проверяем доступность сайта
news = ''
for link in sorted(df2.link,key=lambda _: random.random()):
    if len(news) < 4000:
        req = Request(
            url=link,
            headers={'User-Agent': 'Mozilla/5.0'}
        )
        website_url = urlopen(req).read()
        soup = BeautifulSoup(website_url, 'lxml')

        # Extract the text of the article
        article_inherit = soup.find("div", class_="post-inherit__description").get_text()
        article_body = soup.find("div", class_="forecast-text")

        # Check the last p tags for the presence of a tags
        for p in article_body.find_all("b")[-3:]:
            if p.find("a"):
                p.extract()

        for td in article_body.find_all('p', class_=None):
            news += str(td.text) + ' '
        news = article_inherit+news+'\n'
        news = news.replace('\n', ' ').replace('\t', ' ').replace('   ', ' ').replace('  ', ' ')
print (news)

 19 февраля состоится матч 21-го тура чемпионата Португалии «Каса Пиа» - «Витория Гимарайнш». Согласно представлениям букмекеров, немного выше шансы хозяев поля. Давайте попробуем разобраться в предматчевых раскладах. В 20 турах элитного португальского дивизиона «Каса Пиа» добилась весьма впечатляющих следующих результатов: девять побед, четыре ничьих и семь поражений при общей разнице голов 18-20. Такое выступление привело команду на шестую позицию чемпионата. Этот коллектив увереннее чувствует себя за периметром домашнего стадиона: пять побед, две ничьих и три поражения. Забито 10, пропущено 12. Это четвертый показатель Примейры. И вот так «Каса Пиа» выступает при поддержке своих трибун: четыре победы, две ничьих и четыре поражения при общей разнице голов 8-8. Можно отметить, что в трех последних домашних играх этой команды сыграла ставка «обе забьют и ТБ (2,5)». В преддверии этого противостояния «Гимарайнш» пришвартовался к пятому месту турнирной таблицы Примейры с 33 очками на счет